# **Step 0 - Setup**

In [1]:
# define table functions
from terminaltables import AsciiTable

def table(df):
    table_data = [df.columns.tolist()] + df.values.tolist()
    table = AsciiTable(table_data)
    print(table.table)

def table_h(df):
    # Select the first 15 rows
    df_selected = pd.concat([df.head(15)])
    
    # Prepare the table data: first row is the column names and the rest is the data
    table_data = [df.columns.tolist()] + df_selected.values.tolist()
    
    # Create and print the table
    table = AsciiTable(table_data)
    print(table.table)

def table_s(df):
    # Select the first 4 and last 4 columns
    df_selected = df.iloc[:, :4].join(df.iloc[:, -4:])
    
    # Prepare the table data: first row is the column names and the rest is the data
    table_data = [df_selected.columns.tolist()] + df_selected.values.tolist()
    
    # Create and print the table
    table = AsciiTable(table_data)
    print(table.table)

def table_xs(df):
    # Select the first 3 and last 3 columns
    df_selected = df.iloc[:, :3].join(df.iloc[:, -3:])
    
    # Prepare the table data: first row is the column names and the rest is the data
    table_data = [df_selected.columns.tolist()] + df_selected.values.tolist()
    
    # Create and print the table
    table = AsciiTable(table_data)
    print(table.table)

In [2]:
# define pasteurize function
import re

def pasteurize(df):

    def clean_column_names(col_name):

        cleaned = col_name.lower()
        cleaned = re.sub(r'[^a-z0-9_]', '_', cleaned)
        cleaned = re.sub(r'_+', '_', cleaned)
        cleaned = cleaned.strip('_')
        return cleaned
        
    return (df
            .dropna(how='all')
            .drop_duplicates()
            .rename(columns=clean_column_names)
            .applymap(lambda x: x.strip() if isinstance(x, str) else x))

# **Step 1 - Munge transactions**

**Define a function for processing pitbull transaction files**

In [3]:
import pdfplumber
import re
from typing import Dict, List, Tuple
import pandas as pd
from pathlib import Path

def extract_tax_info(pdf_path: str) -> Tuple[Dict[str, str], List[Tuple[str, str]]]:
    """
    Extract specific tax information from a PDF file.
    
    Args:
        pdf_path: Path to the PDF file
    
    Returns:
        Tuple containing:
        - Dictionary with taxpayer info (name and EIN)
        - List of tuples with (period, refunds) for each row
    """
    # Initialize data structures
    taxpayer_info = {}
    period_refunds = []
    
    with pdfplumber.open(pdf_path) as pdf:
        # Get text from first page
        text = pdf.pages[2].extract_text()
        
        # Extract taxpayer information using regex
        name_match = re.search(r'Taxpayer Name: (.+?)(?:\n|Taxpayer)', text)
        ein_match = re.search(r'Taxpayer EIN: (.+?)(?:\n|$)', text)
        
        if name_match:
            taxpayer_info['name'] = name_match.group(1).strip()
        if ein_match:
            taxpayer_info['ein'] = ein_match.group(1).strip()
        
        # Extract table data
        lines = text.split('\n')
        for line in lines:
            if re.match(r'\d{2}/\d{4}', line):
                columns = line.split()
                period = columns[0]
                refunds = columns[-4] if len(columns) >= 4 else ''
                
                period_refunds.append((period, refunds))
    
    return taxpayer_info, period_refunds

def process_pdf_folder(folder_path: str) -> pd.DataFrame:
    """
    Process all PDFs in a folder and return results as a DataFrame.
    
    Args:
        folder_path: Path to folder containing PDFs
    
    Returns:
        pandas DataFrame with extracted information
    """
    # Initialize lists to store data
    all_data = []
    
    # Get all PDF files in the folder
    pdf_files = Path(folder_path).glob('*.pdf')
    
    for pdf_path in pdf_files:
        try:
            taxpayer_info, period_refunds = extract_tax_info(str(pdf_path))
            
            # Create a row for each period/refund combination
            for period, refund in period_refunds:
                row_data = {
                    'file_name': pdf_path.name,
                    'taxpayer_name': taxpayer_info.get('name', ''),
                    'taxpayer_ein': taxpayer_info.get('ein', ''),
                    'period': period,
                    'refunds': refund
                }
                all_data.append(row_data)
                
        except Exception as e:
            print(f"Error processing {pdf_path.name}: {str(e)}")
            continue
    
    # Create DataFrame
    df = pd.DataFrame(all_data)
    
    # Clean up refunds column (remove parentheses and convert to numeric)
    df['refunds'] = df['refunds'].str.replace('[$(),]', '', regex=True)
    df['refunds'] = pd.to_numeric(df['refunds'], errors='coerce')
    
    return df

In [ ]:
if __name__ == "__main__":
    folder_path = './pitbull/01-30-25' # Replace with your folder path
    
    try:
        # Process all PDFs and create DataFrame
        df = process_pdf_folder(folder_path)
        
        # Display first few rows
        print("\nFirst few rows of the DataFrame:")
        print(df.head())
        
        # Basic statistics
        print("\nDataFrame Info:")
        print(df.info())
        
        # Optionally save to CSV
        # df.to_csv('tax_data.csv', index=False)
        
    except Exception as e:
        print(f"Error: {str(e)}")

**Create quarter_year column**

In [5]:
import coconut
%load_ext coconut

In [7]:
 %%coconut

from siuba import *

df = df |> mutate(
    quarter_year = case_when({
        _.period == '06/2020': 'Q2_2020', 
        _.period == '09/2020': 'Q3_2020', 
        _.period == '12/2020': 'Q4_2020',
        _.period == '03/2021': 'Q1_2021',
        _.period == '06/2021': 'Q2_2021',
        _.period == '09/2021': 'Q3_2021',
        _.period == '12/2021': 'Q4_2021'
    })
) 

df = df |> select(_.taxpayer_name, 
            _.taxpayer_ein, 
            _.period, 
            _.quarter_year, 
            _.refunds)

df |> select (-_.taxpayer_name) |> table_h

+--------------+---------+--------------+----------+
| taxpayer_ein | period  | quarter_year | refunds  |
+--------------+---------+--------------+----------+
| XX-XXX5545   | 06/2020 | Q2_2020      | 35115.6  |
| XX-XXX5545   | 09/2020 | Q3_2020      | 61591.56 |
| XX-XXX5545   | 12/2020 | Q4_2020      | 0.0      |
| XX-XXX5545   | 03/2021 | Q1_2021      | 63341.07 |
| XX-XXX5545   | 06/2021 | Q2_2021      | 0.0      |
| XX-XXX5545   | 09/2021 | Q3_2021      | 0.0      |
| XX-XXX5545   | 12/2021 | Q4_2021      | 0.0      |
| XX-XXX5545   | 06/2020 | Q2_2020      | 35115.6  |
| XX-XXX5545   | 09/2020 | Q3_2020      | 61591.56 |
| XX-XXX5545   | 12/2020 | Q4_2020      | 0.0      |
| XX-XXX5545   | 03/2021 | Q1_2021      | 63341.07 |
| XX-XXX5545   | 06/2021 | Q2_2021      | 0.0      |
| XX-XXX5545   | 09/2021 | Q3_2021      | 0.0      |
| XX-XXX5545   | 12/2021 | Q4_2021      | 0.0      |
| XX-XXX7945   | 06/2020 | Q2_2020      | 0.0      |
+--------------+---------+--------------+-----

**Define function for pivoting on quarters**

In [8]:
def create_quarter_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Creates a wide-format dataframe with separate columns for each quarter's refunds.
    
    Args:
        df: DataFrame with columns [taxpayer_name, taxpayer_ein, quarter_year, refunds]
    
    Returns:
        DataFrame with a column for each quarter's refund amount
    """
    # Create refund pivot
    refund_pivot = pd.pivot_table(
        data=df,
        index=['taxpayer_name', 'taxpayer_ein'],
        columns='quarter_year',
        values='refunds',
        fill_value=pd.NA
    )
    
    # Add prefix to column names
    refund_pivot.columns = [f'refunds_{col}' for col in refund_pivot.columns]
    
    # Reset index to make taxpayer_name and taxpayer_ein regular columns, and fill NANs with 0s
    final_df = refund_pivot.reset_index().fillna(0)
    
    return final_df

In [10]:
%%coconut

# apply the pivot function and create a new column for tracking the total refunds for each quarter
transactions = df.copy() \
    |> create_quarter_columns \
    |> mutate(
        refund_total = 
        _.refunds_Q2_2020.fillna(0) + 
        _.refunds_Q3_2020.fillna(0) + 
        _.refunds_Q4_2020.fillna(0) + 
        _.refunds_Q1_2021.fillna(0) + 
        _.refunds_Q2_2021.fillna(0) + 
        _.refunds_Q3_2021.fillna(0) + 
        _.refunds_Q4_2021.fillna(0)
    ) |> _.round(2)

# also add a column for the tin trail
transactions = transactions |> mutate(
    tin_trail = _.taxpayer_ein.apply(lambda x: 
            re.search(r'XX-XXX(\d{4})', x).group(1) if pd.notna(x) else None)
)

transactions |> select(-_.taxpayer_name) |> head(15) |> table

+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+--------------+-----------+
| taxpayer_ein | refunds_Q1_2021 | refunds_Q2_2020 | refunds_Q2_2021 | refunds_Q3_2020 | refunds_Q3_2021 | refunds_Q4_2020 | refunds_Q4_2021 | refund_total | tin_trail |
+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+--------------+-----------+
| XX-XXX5545   | 63341.07        | 35115.6         | 0.0             | 61591.56        | 0.0             | 0.0             | 0.0             | 160048.23    | 5545      |
| XX-XXX7945   | 0.0             | 0.0             | 0.0             | 0.0             | 2641.57         | 0.0             | 0.0             | 2641.57      | 7945      |
| XX-XXX1523   | 0.0             | 7860.09         | 0.0             | 20082.07        | 0.0             | 26859.89        | 0.0             | 54802.0

**Load in client list**

In [12]:
%%coconut

import re

clients = pd.read_csv('transcript-log.csv') \
    |> pasteurize \
    |> select(~_.quarters_filed) \
    |> mutate(
        tin_trail = _.tin.apply(lambda x: 
            re.search(r'\d{2}-\d{3}(\d{4})', x).group(1) if pd.notna(x) else None)
    )

clients |> select(_.tin_trail) |> head(15) |> table 

+-----------+
| tin_trail |
+-----------+
| 5217      |
| 7485      |
| 8326      |
| 6167      |
| 5252      |
| 1055      |
| 7829      |
| 1098      |
| 8142      |
| 5545      |
| 4890      |
| 4990      |
| 8731      |
| 8728      |
| 6160      |
+-----------+


**Join the transactions with the client list**

In [14]:
%%coconut

df_joined = transactions.merge(
    clients,
    how='left',
    on='tin_trail'
)

df_joined = df_joined |> select('taxpayer_name', 
                                'tin', 
                                'refund_total',
                                'refunds_Q2_2020', 
                                'refunds_Q3_2020', 
                                'refunds_Q4_2020',
                                'refunds_Q1_2021', 
                                'refunds_Q2_2021',
                                'refunds_Q3_2021',
                                'refunds_Q4_2021'
                               )

df_joined |> select(-_.taxpayer_name, 
                    -_.tin) \
            |> head(15) \
            |> table

+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
| refund_total | refunds_Q2_2020 | refunds_Q3_2020 | refunds_Q4_2020 | refunds_Q1_2021 | refunds_Q2_2021 | refunds_Q3_2021 | refunds_Q4_2021 |
+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
| 160048.23    | 35115.6         | 61591.56        | 0.0             | 63341.07        | 0.0             | 0.0             | 0.0             |
| 2641.57      | 0.0             | 0.0             | 0.0             | 0.0             | 0.0             | 2641.57         | 0.0             |
| 54802.05     | 7860.09         | 20082.07        | 26859.89        | 0.0             | 0.0             | 0.0             | 0.0             |
| 11136.81     | 4297.54         | 4027.46         | 0.0             | 2811.81         | 0.0             | 0.0             | 0.0             |

In [12]:
# export the joined table
df_joined.to_csv('transcations-processed-01-30.csv', index = False)

In [16]:
%%coconut

# read in the joined table
transactions_proc = pd.read_csv('transcations-processed-01-30.csv')

transactions_proc |> select(-_.taxpayer_name, -_.tin) |> table_h

+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
| refund_total | refunds_Q2_2020 | refunds_Q3_2020 | refunds_Q4_2020 | refunds_Q1_2021 | refunds_Q2_2021 | refunds_Q3_2021 | refunds_Q4_2021 |
+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
| 160048.23    | 35115.6         | 61591.56        | 0.0             | 63341.07        | 0.0             | 0.0             | 0.0             |
| 2641.57      | 0.0             | 0.0             | 0.0             | 0.0             | 0.0             | 2641.57         | 0.0             |
| 54802.05     | 7860.09         | 20082.07        | 26859.89        | 0.0             | 0.0             | 0.0             | 0.0             |
| 11136.81     | 4297.54         | 4027.46         | 0.0             | 2811.81         | 0.0             | 0.0             | 0.0             |

# **Step 2 - Munge 941-x forms for EIN, year and quarter**

In [17]:
%%coconut

from pathlib import Path 
import pdfplumber

folder_path = Path('./941s')

def extract_text_from_pdf(pdf_path):
    all_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            all_text.append(text)
    return '\n'.join(all_text)

# Use with:
files = folder_path.glob('*.pdf')

# Check:
print(f'Files found: {files |> list |> len}')
for file in files:
    print(file.name)

Files found: 330


In [18]:
# For EIN - handles both formats:
# 1. "12 – 3456789" (connected)
# 2. "12 3456789" (split across lines with optional dash)
ein_pattern = r'(\d{2}\s*[–-]\s*\d{7})'
# ein_pattern = r'(\d{2}\s*[–-]?\s*\d{7})'

# For quarter (single digit after 'quarter')
quarter_pattern = r'Check the ONE quarter.+?(\d)'
# quarter_pattern = r'quarter\s+(\d)'

# For year - handle both:
# 1. "2020" (on same line)
# 2. "– 2021" (on next line)
year_pattern = r'(?:calendar year|YYYY)[^0-9]*(\d{4})'
# year_pattern = r'(?:\d{7}|–)\s*(\d{4})'

In [19]:
import re

# Test case 1
text1 = """Name (not your trade name) Employer identification number (EIN) Correcting quarter 2 (1, 2, 3, 4)
Correcting calendar year (YYYY)
[       ], LLC [      ] 12 – 3456789 2020"""

# Test case 2
text2 = """Name (not your trade name) Employer identification number (EIN) Correcting quarter 1 (1, 2, 3, 4)
Correcting calendar year (YYYY)
[         ] 12 3456789
– 2021"""

def extract_info(text):
    ein = re.search(ein_pattern, text)
    quarter = re.search(quarter_pattern, text)
    year = re.search(year_pattern, text)
    
    return {
        'ein': ein.group(1) if ein else None,
        'quarter': quarter.group(1) if quarter else None,
        'year': year.group(1) if year else None
    }

print("Test Case 1:")
print(extract_info(text1))
print("\nTest Case 2:")
print(extract_info(text2))

Test Case 1:
{'ein': '12 – 3456789', 'quarter': None, 'year': None}

Test Case 2:
{'ein': None, 'quarter': None, 'year': None}


**Define a function for extracing EIN, year & quarter**

In [ ]:
def process_page(page):
    text = page.extract_text()
    
    # Print first chunk of text for debugging
    print("First 500 characters of extracted text:")
    print(text[:500])
    print("\n-------------------\n")
    
    # Modified patterns
    ein_pattern = r'\b(\d{2}\s*[–-]\s*\d{7})\b'  # Added word boundaries
    quarter_pattern = r'(?:Correcting\s+quarter|quarter)\s*(\d)'  # Made more flexible
    year_pattern_1 = r'\b\d{2}\s*[–-]\s*\d{7}\s+(\d{4})\b'
    year_pattern_2 = r'(?:calendar\s+year|year).*?(\d{4})'
    
    # Extract info using our patterns
    ein = re.search(ein_pattern, text)
    quarter = re.search(quarter_pattern, text)
    
    # Try both year patterns
    year = None
    year_match_1 = re.search(year_pattern_1, text)
    year_match_2 = re.search(year_pattern_2, text, re.DOTALL | re.IGNORECASE)
    
    print("Matches found:")
    print(f"EIN match: {ein.group(1) if ein else None}")
    print(f"Quarter match: {quarter.group(1) if quarter else None}")
    print(f"Year match 1: {year_match_1.group(1) if year_match_1 else None}")
    print(f"Year match 2: {year_match_2.group(1) if year_match_2 else None}")
    
    if year_match_1:
        year = year_match_1.group(1)
    elif year_match_2:
        year = year_match_2.group(1)
    
    return {
        'ein': ein.group(1) if ein else None,
        'quarter': quarter.group(1) if quarter else None,
        'year': year
    }

# Test on your specific file
with pdfplumber.open("./941s/Go Blue Q1 2021.pdf") as pdf:
    info = process_page(pdf.pages[1])
    print("\nFinal extracted info:")
    print(info)

**Define a function for processing all good 941s**

In [21]:
from pathlib import Path
import pdfplumber
import pandas as pd
import re

def process_page(page):
    text = page.extract_text()
    
    # Modified patterns
    ein_pattern = r'\b(\d{2}\s*[–-]\s*\d{7})\b'  # Added word boundaries
    quarter_pattern = r'(?:Correcting\s+quarter|quarter)\s*(\d)'  # Made more flexible
    year_pattern_1 = r'\b\d{2}\s*[–-]\s*\d{7}\s+(\d{4})\b'
    year_pattern_2 = r'(?:calendar\s+year|year).*?(\d{4})'
    
    # Extract info using our patterns
    ein = re.search(ein_pattern, text)
    quarter = re.search(quarter_pattern, text)
    
    # Try both year patterns
    year = None
    year_match_1 = re.search(year_pattern_1, text)
    year_match_2 = re.search(year_pattern_2, text, re.DOTALL | re.IGNORECASE)
    
    if year_match_1:
        year = year_match_1.group(1)
    elif year_match_2:
        year = year_match_2.group(1)
    
    return {
        'ein': ein.group(1) if ein else None,
        'quarter': quarter.group(1) if quarter else None,
        'year': year
    }

def process_941_pdf(pdf_path):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Get info from second page using our improved process_page function
            page_info = process_page(pdf.pages[1])
            
            return {
                'file_name': pdf_path.name,
                **page_info  # Unpack the dictionary returned by process_page
            }
    except Exception as e:
        print(f"Error processing {pdf_path.name}: {str(e)}")
        return None

def is_941_form(pdf_path):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Check first few pages for '941'
            for page in pdf.pages[:2]:  # Check first 2 pages
                text = page.extract_text()
                if '941' in text:
                    return True
        return False
    except Exception as e:
        print(f"Error checking {pdf_path.name}: {str(e)}")
        return False

def process_all_941s(folder_path):
    all_records = []
    folder = Path(folder_path)
    
    # Get all PDFs
    pdfs = folder.glob('*.pdf')
    
    for pdf_path in pdfs:
        # First check if it's a 941 form
        if is_941_form(pdf_path):
            record = process_941_pdf(pdf_path)
            if record:
                all_records.append(record)
    
    return pd.DataFrame(all_records)

**Process all of the good 941 forms**

In [ ]:
# Process files in your 941s folder
df = process_all_941s("./941s")

# Look at the results
print("\nOverview of data:")
print(df.info())

print("\nSample of records:")
print(df.head())

print("\nCheck for missing values:")
print(df.isnull().sum())

In [ ]:
%%coconut

# preview the good 941 df
from siuba import mutate, _, filter as siuba_filter

df_941s_good = df.copy() \
    |> siuba_filter(_.ein.notna()) \
    |> mutate(ein = _.ein \ 
        |> .str.replace(r' \– ', '-', regex=True) \ 
        |> .str.replace(r' ', '-', regex = True))

df_941s_good \
    |> select(-_.file_name) \
    |> mutate(ein = _.ein.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \
    |> head(15) \
    |> table

+------------+---------+------+
| ein        | quarter | year |
+------------+---------+------+
| XX-XXX4548 | 2       | 2020 |
| XX-XXX4548 | 3       | 2020 |
| XX-XXX4548 | 4       | 2020 |
| XX-XXX4548 | 1       | 2021 |
| XX-XXX4548 | 2       | 2021 |
| XX-XXX4548 | 3       | 2021 |
| XX-XXX8728 | 2       | 2020 |
| XX-XXX8728 | 3       | 2020 |
| XX-XXX8728 | 4       | 2020 |
| XX-XXX8728 | 1       | 2021 |
| XX-XXX8728 | 2       | 2021 |
| XX-XXX8728 | 2       | 2021 |
| XX-XXX8332 | 4       | 2020 |
| XX-XXX8332 | 3       | 2021 |
| XX-XXX1098 | 1       | 2021 |
+------------+---------+------+


**Export and read back in the good 941s df**

In [ ]:
# save the 941 dataframe!
df_941s_good.to_csv('good_941s.csv', index = False)

In [30]:
%%coconut

# read back in the 941 dataframe
df_941s_good = pd.read_csv('good_941s.csv')

df_941s_good \
    |> select(-_.file_name) \
    |> mutate(ein = _.ein.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \
    |> table

+------------+---------+------+
| ein        | quarter | year |
+------------+---------+------+
| XX-XXX4548 | 2       | 2020 |
| XX-XXX4548 | 3       | 2020 |
| XX-XXX4548 | 4       | 2020 |
| XX-XXX4548 | 1       | 2021 |
| XX-XXX4548 | 2       | 2021 |
| XX-XXX4548 | 3       | 2021 |
| XX-XXX8728 | 2       | 2020 |
| XX-XXX8728 | 3       | 2020 |
| XX-XXX8728 | 4       | 2020 |
| XX-XXX8728 | 1       | 2021 |
| XX-XXX8728 | 2       | 2021 |
| XX-XXX8728 | 2       | 2021 |
| XX-XXX8332 | 4       | 2020 |
| XX-XXX8332 | 3       | 2021 |
| XX-XXX1098 | 1       | 2021 |
| XX-XXX1098 | 2       | 2020 |
| XX-XXX1098 | 2       | 2021 |
| XX-XXX1098 | 3       | 2020 |
| XX-XXX1098 | 3       | 2021 |
| XX-XXX1098 | 4       | 2020 |
| XX-XXX2498 | 1       | 2021 |
| XX-XXX2498 | 2       | 2020 |
| XX-XXX2498 | 2       | 2021 |
| XX-XXX2498 | 3       | 2020 |
| XX-XXX2498 | 2       | 2021 |
| XX-XXX7059 | 1       | 2021 |
| XX-XXX7059 | 2       | 2021 |
| XX-XXX7059 | 3       | 2021 |
| XX-XXX

**Define a function to process bad 941s**

In [31]:
import cv2
import numpy as np
import pytesseract
from pdf2image import convert_from_path
import re

class Form941XCoordinates:
    def __init__(self, width, height):
        # Format: (x, y, width, height) - for the header section only
        self.ein_first = (2685, 440, 160, 120)          # First part of EIN (2 digits)
        self.ein_second = (3235, 440, 375, 120)         # Second part of EIN (7 digits)
        self.quarter_box = (4385, 305, 130, 95)         # Quarter checkbox
        self.year = (4210, 500, 250, 80)                # Year field

def visualize_regions(image, coords):
    """Draw colored rectangles around the regions we're trying to extract."""
    debug_image = image.copy()
    
    # Define regions with their colors
    regions = [
        (coords.ein_first, "EIN First", (0, 0, 255)),      # Red
        (coords.ein_second, "EIN Second", (0, 255, 0)),    # Green
        (coords.quarter_box, "Quarter", (255, 0, 0)),      # Blue
        (coords.year, "Year", (255, 255, 0))              # Cyan
    ]
    
    # Draw rectangles and labels
    for (x, y, w, h), label, color in regions:
        cv2.rectangle(debug_image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(debug_image, label, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    cv2.imwrite('debug_visualization.png', debug_image)
    return debug_image

def process_quarter_roi(roi):
    """Enhanced processing specifically for quarter checkbox region"""
    # Apply stronger contrast enhancement
    roi = cv2.convertScaleAbs(roi, alpha=2.0, beta=-50)
    
    # Apply adaptive thresholding instead of global
    roi = cv2.adaptiveThreshold(
        roi,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        11,
        2
    )
    
    # Clean up noise
    kernel = np.ones((2,2), np.uint8)
    roi = cv2.morphologyEx(roi, cv2.MORPH_CLOSE, kernel)
    
    # Invert back for OCR
    roi = cv2.bitwise_not(roi)
    
    return roi

def validate_ein_part(ein_str, expected_length):
    """Validate and format EIN parts."""
    # Handle None or empty string
    if not ein_str:
        return ''
        
    # Remove any non-digit characters
    digits = ''.join([char for char in str(ein_str) if char.isdigit()])
    
    if not digits:
        return ''
        
    # If we get a single digit for the first part where we expect two,
    # check if it might be a repeated digit (like '77' being read as '7')
    if expected_length == 2 and len(digits) == 1:
        digits = digits + digits  # Duplicate the digit
    
    # Pad with leading zeros if necessary
    return digits.zfill(expected_length)

def extract_941x_data(pdf_path):
    """Extract key information from Form 941-X header using targeted OCR."""
    print("\nStarting extraction...")
    
    pages = convert_from_path(pdf_path, dpi=600)
    full_image = np.array(pages[1])
    header_image = full_image[0:900, :]
    header_image_bgr = cv2.cvtColor(header_image, cv2.COLOR_RGB2BGR)
    
    # Save the header image for inspection
    cv2.imwrite('header_original.png', header_image_bgr)
    
    gray = cv2.cvtColor(header_image, cv2.COLOR_RGB2GRAY)
    
    coords = Form941XCoordinates(header_image.shape[1], header_image.shape[0])
    
    # Create and save visualization
    debug_image = visualize_regions(header_image_bgr, coords)
    
    # Extract ROIs
    def get_roi(coords_tuple, name):
        x, y, w, h = coords_tuple
        roi = gray[y:y+h, x:x+w]
        padded = cv2.copyMakeBorder(roi, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=255)
        cv2.imwrite(f'roi_{name}.png', padded)
        return padded

    ein_first_roi = get_roi(coords.ein_first, "ein_first")
    ein_second_roi = get_roi(coords.ein_second, "ein_second")
    quarter_roi = get_roi(coords.quarter_box, "quarter")
    year_roi = get_roi(coords.year, "year")
    
    def threshold_roi(roi):
        roi = cv2.GaussianBlur(roi, (3,3), 0)
        roi = cv2.convertScaleAbs(roi, alpha=1.5, beta=0)
        _, roi = cv2.threshold(roi, 127, 255, cv2.THRESH_BINARY)
        return roi
    
    # Process and extract text from ROIs with enhanced EIN processing
    ein_first = pytesseract.image_to_string(
        threshold_roi(ein_first_roi),
        config='--psm 7 -c tessedit_char_whitelist=0123456789'
    ).strip()
    
    ein_first = validate_ein_part(ein_first, 2)  # Should be 2 digits
    
    ein_second = pytesseract.image_to_string(
        threshold_roi(ein_second_roi),
        config='--psm 7 -c tessedit_char_whitelist=0123456789'
    ).strip()
    
    ein_second = validate_ein_part(ein_second, 7)  # Should be 7 digits
    
    # Enhanced quarter processing
    quarter_processed = process_quarter_roi(quarter_roi)
    cv2.imwrite('quarter_processed.png', quarter_processed)  # Save for inspection
    
    quarter = pytesseract.image_to_string(
        quarter_processed,
        config='--psm 7 -c tessedit_char_whitelist=1234'
    ).strip()
    
    # Try alternative quarter detection if first attempt fails
    if not quarter:
        # Check for marked checkbox using pixel density
        quarter_binary = cv2.threshold(quarter_roi, 127, 255, cv2.THRESH_BINARY_INV)[1]
        regions = np.hsplit(quarter_binary, 4)  # Split into 4 regions
        densities = [np.sum(region) / 255 for region in regions]
        max_density_idx = np.argmax(densities)
        if densities[max_density_idx] > 100:  # Threshold for considering a box checked
            quarter = str(max_density_idx + 1)
    
    year = pytesseract.image_to_string(
        threshold_roi(year_roi),
        config='--psm 7 -c tessedit_char_whitelist=0123456789'
    ).strip()
    
    print(f"Raw EIN parts: '{ein_first}' - '{ein_second}'")
    print(f"Quarter: '{quarter}'")
    print(f"Year: '{year}'")
    
    return {
        "ein": f"{ein_first}-{ein_second}" if ein_first and ein_second else "",
        "quarter": quarter if quarter else "",
        "year": year if year else ""
    }

def process_941x_form(pdf_path):
    """Process a Form 941-X and return extracted data."""
    try:
        data = extract_941x_data(pdf_path)
        print("\nExtracted data:", data)
        return data
    except Exception as e:
        print(f"Processing error: {str(e)}")
        return None

In [ ]:
# test the processing of a bad 941x form
result = process_941x_form("./941s/KCJ Q4 2020.pdf")

print(result)

**Process all of the bad 941s!**

In [ ]:
import os
import pandas as pd
from pathlib import Path

def process_folder(folder_path, df2):
    """
    Process all PDFs in the specified folder that meet the criteria:
    - Not in df2.file_name
    - Don't contain '2848', '8821', or '943' in filename
    
    Args:
        folder_path (str): Path to folder containing PDFs
        df2 (pd.DataFrame): DataFrame containing file_name column of files to exclude
    
    Returns:
        list: List of dictionaries containing processed results
    """
    # Convert folder path to Path object
    folder = Path(folder_path)
    
    # Get list of excluded filenames from df2
    excluded_files = set(df_941s_good['file_name'].tolist())
    
    # Get all PDF files in the folder
    results = []
    
    for pdf_file in folder.glob('*.pdf'):
        # Skip if filename contains 2848 or 8821
        if '2848' in pdf_file.name or '8821' in pdf_file.name or '943' in pdf_file.name:
            print(f"Skipping {pdf_file.name} - contains excluded form number")
            continue
            
        # Skip if file is in df2
        if pdf_file.name in excluded_files:
            print(f"Skipping {pdf_file.name} - already processed")
            continue
            
        print(f"\nProcessing {pdf_file.name}...")
        try:
            # Process the file using existing function
            result = process_941x_form(str(pdf_file))
            if result:
                results.append({
                    'file_name': pdf_file.name,
                    'ein': result['ein'],
                    'quarter': result['quarter'],
                    'year': result['year']
                })
        except Exception as e:
            print(f"Error processing {pdf_file.name}: {str(e)}")
            continue
    
    # Convert results to DataFrame
    df_941s_bad = pd.DataFrame(results)
    
    # Print summary
    print("\nProcessing complete!")
    print(f"Total files processed: {len(results)}")
    
    return df_941s_bad

if __name__ == "__main__":
    # Example usage
    folder_path = "./941s"
    df2 = pd.DataFrame({'file_name': ['already_processed.pdf']})  # Example df2
    
    df_941s_bad = process_folder(folder_path, df2)
    print("\nResults:")

In [40]:
%%coconut

# preview the bad 941s df
df_941s_bad \
    |> select(-_.file_name) \
    |> mutate(ein = _.ein.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \
    |> head(15) \
    |> table

+------------+---------+------+
| ein        | quarter | year |
+------------+---------+------+
| XX-XXX8326 | 2       | 2020 |
| XX-XXX8326 | 3       | 2020 |
| XX-XXX8326 | 4       | 2020 |
| XX-XXX8142 | 1       | 2021 |
| XX-XXX8142 | 2       | 2020 |
| XX-XXX8142 | 2       | 2021 |
| XX-XXX8142 | 3       | 2020 |
| XX-XXX8142 | 3       | 2021 |
| XX-XXX8142 | 4       | 2020 |
| XX-XXX3863 | 2       | 2021 |
| XX-XXX3863 | 3       | 2020 |
| XX-XXX3863 | 3       | 2021 |
| XX-XXX3863 | 4       | 2020 |
| XX-XXX8326 | 1       | 2021 |
| XX-XXX8326 | 2       | 2021 |
+------------+---------+------+


**Export and read back in the good 941s df**

In [ ]:
# save the 941 dataframe
df_941s_bad.to_csv('bad_941s', index = False)

In [42]:
%%coconut

# read back in the 941 dataframe
df_941s_bad = pd.read_csv('bad_941s.csv')

df_941s_bad \
    |> select(-_.file_name) \
    |> mutate(ein = _.ein.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \
    |> head(15) \
    |> table

+------------+---------+------+
| ein        | quarter | year |
+------------+---------+------+
| XX-XXX8326 | 2       | 2020 |
| XX-XXX8326 | 3       | 2020 |
| XX-XXX8326 | 4       | 2020 |
| XX-XXX8142 | 1       | 2021 |
| XX-XXX8142 | 2       | 2020 |
| XX-XXX8142 | 2       | 2021 |
| XX-XXX8142 | 3       | 2020 |
| XX-XXX8142 | 3       | 2021 |
| XX-XXX8142 | 4       | 2020 |
| XX-XXX3863 | 2       | 2021 |
| XX-XXX3863 | 3       | 2020 |
| XX-XXX3863 | 3       | 2021 |
| XX-XXX3863 | 4       | 2020 |
| XX-XXX8326 | 1       | 2021 |
| XX-XXX8326 | 2       | 2021 |
+------------+---------+------+


**Combine the good and bad pdf dataframes**

In [60]:
%%coconut

from siuba import \filter

df_941s_combined = pd.concat([df_941s_good, df_941s_bad], ignore_index = True, axis = 0) \
    |> arrange(_.file_name) \
    |> filter(_.ein != '')

df_941s_combined \
    |> select(-_.file_name) \
    |> mutate(ein = _.ein.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \
    |> head(15) \
    |> table

+------------+---------+------+
| ein        | quarter | year |
+------------+---------+------+
| XX-XXX4548 | 2       | 2020 |
| XX-XXX4548 | 3       | 2020 |
| XX-XXX4548 | 4       | 2020 |
| XX-XXX4548 | 1       | 2021 |
| XX-XXX4548 | 2       | 2021 |
| XX-XXX4548 | 3       | 2021 |
| XX-XXX8728 | 2       | 2020 |
| XX-XXX8728 | 3       | 2020 |
| XX-XXX8728 | 4       | 2020 |
| XX-XXX8728 | 1       | 2021 |
| XX-XXX8728 | 2       | 2021 |
| XX-XXX8728 | 2       | 2021 |
| XX-XXX8142 | 1       | 2021 |
| XX-XXX8142 | 2       | 2020 |
| XX-XXX8142 | 2       | 2021 |
+------------+---------+------+


# **Step 3 - Munge 941-x forms for line 27 values**

**Define a function for extracting line 27 values**

In [46]:
import cv2
import numpy as np
import pytesseract
from pdf2image import convert_from_path
import re

class Form941XCoordinates:
    def __init__(self, width, height):
        # Format: (x, y, width, height) - for the header section only
        self.line_27 = (155, 55, 590, 110)          

def visualize_regions(image, coords):
    """Draw colored rectangles around the regions we're trying to extract."""
    debug_image = image.copy()
    
    # Define regions with their colors
    regions = [
        (coords.line_27, "Line 27 :-)", (255, 255, 0))              # Cyan
    ]
    
    # Draw rectangles and labels
    for (x, y, w, h), label, color in regions:
        cv2.rectangle(debug_image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(debug_image, label, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    cv2.imwrite('debug_visualization.png', debug_image)
    return debug_image

def extract_941x_data(pdf_path):
    """Extract line 27 from Form 941-X header using targeted OCR."""
    print("\nStarting extraction...")
    
    # Convert first page of PDF to image with higher DPI
    pages = convert_from_path(pdf_path, dpi=600)
    full_image = np.array(pages[2])
    
    # Crop just the header section (top 900 pixels)
    header_image = full_image[3185:3760, 4050:4950]
    header_image_bgr = cv2.cvtColor(header_image, cv2.COLOR_RGB2BGR)
    
    # Convert to grayscale for OCR
    gray = cv2.cvtColor(header_image, cv2.COLOR_RGB2GRAY)
    
    # Create coordinates based on image size
    coords = Form941XCoordinates(header_image.shape[1], header_image.shape[0])
    
    # Create and save visualization
    debug_image = visualize_regions(header_image_bgr, coords)
    
    # Extract ROIs
    def get_roi(coords_tuple, name):
        x, y, w, h = coords_tuple
        roi = gray[y:y+h, x:x+w]
        padded = cv2.copyMakeBorder(roi, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=255)
        cv2.imwrite(f'roi_{name}.png', padded)
        return padded

    line_27_roi = get_roi(coords.line_27, "line_27")
    
    def threshold_roi(roi):
        # Enhanced preprocessing
        roi = cv2.GaussianBlur(roi, (3,3), 0)
        roi = cv2.convertScaleAbs(roi, alpha=1.5, beta=0)
        _, roi = cv2.threshold(roi, 127, 255, cv2.THRESH_BINARY)
        return roi
    
    # Process and extract text from each ROI
    line_27 = pytesseract.image_to_string(
        threshold_roi(line_27_roi),
        config='--psm 7 -c tessedit_char_whitelist=0123456789'
    ).strip()
        
    print(f"Line 27: '{line_27}'")
    
    return {
        "line 27": line_27 if line_27 else ""
    }

def process_line_27(pdf_path):
    """Process a Form 941-X and return extracted line 27."""
    try:
        data = extract_941x_data(pdf_path)
        print("\nExtracted data:", data)
        return data
    except Exception as e:
        print(f"Processing error: {str(e)}")
        return None

In [ ]:
# test the function
result = process_line_27("./941s/NSN Q1 2021 v2-70152ddbc1fd4c9da15829b4e5efad00897402e7.pdf")

print(result)

**Extract all the line 27 values!**

In [ ]:
import os
import pandas as pd
from pathlib import Path

def process_folder(folder_path):
    """
    Process all PDFs in the specified folder that meet the criteria:
    - Don't contain '2848', '8821', or '943' in filename
    
    Args:
        folder_path (str): Path to folder containing PDFs
    
    Returns:
        pd.DataFrame: DataFrame containing processed results
    """
    # Convert folder path to Path object
    folder = Path(folder_path)
    
    # Get all PDF files in the folder
    results = []
    
    for pdf_file in folder.glob('*.pdf'):
        # Skip if filename contains excluded form numbers
        if any(form in pdf_file.name for form in ['2848', '8821', '943']):
            print(f"Skipping {pdf_file.name} - contains excluded form number")
            continue
            
        print(f"\nProcessing {pdf_file.name}...")
        try:
            # Process the file using the line 27 extraction function
            result = process_line_27(str(pdf_file))
            if result:
                results.append({
                    'file_name': pdf_file.name,
                    'line_27': result['line 27']
                })
                print(f"Successfully processed {pdf_file.name}")
        except Exception as e:
            print(f"Error processing {pdf_file.name}: {str(e)}")
            continue
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    # Print summary
    print("\nProcessing complete!")
    print(f"Total files processed: {len(results)}")
    
    return results_df

if __name__ == "__main__":
    # Example usage
    df_line27 = process_folder("./941s")
    print("\nResults:")
    print(df_line27)

In [55]:
%%coconut

df_line27 \
     |> select(-_.file_name) \
     |> _.reset_index() \
     |> head(15) \
     |> table

+-------+---------+
| index | line_27 |
+-------+---------+
| 0     | 646831  |
| 1     | 791070  |
| 2     | 578313  |
| 3     | 842800  |
| 4     | 1773100 |
| 5     | 2186100 |
| 6     | 866721  |
| 7     | 1027442 |
| 8     | 377358  |
| 9     | 2032059 |
| 10    | 816728  |
| 11    | 654438  |
| 12    | 3235630 |
| 13    | 4877512 |
| 14    | 4139199 |
+-------+---------+


# **Step 4 - Join all 941-x dataframes**

**Join the combined 431 table with the new line 27 table**

In [63]:
%%coconut

imperfect_941 = inner_join(df_941s_combined, df_line27, 
                    on = 'file_name')

imperfect_941 \
    |> select(-_.file_name) \
    |> mutate(ein = _.ein.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \
    |> head(15) \
    |> table

+------------+---------+------+---------+
| ein        | quarter | year | line_27 |
+------------+---------+------+---------+
| XX-XXX4548 | 2       | 2020 | 646831  |
| XX-XXX4548 | 3       | 2020 | 791070  |
| XX-XXX4548 | 4       | 2020 | 578313  |
| XX-XXX4548 | 1       | 2021 | 842800  |
| XX-XXX4548 | 2       | 2021 | 1773100 |
| XX-XXX4548 | 3       | 2021 | 2186100 |
| XX-XXX8728 | 2       | 2020 | 2032059 |
| XX-XXX8728 | 3       | 2020 | 816728  |
| XX-XXX8728 | 4       | 2020 | 654438  |
| XX-XXX8728 | 1       | 2021 | 3235630 |
| XX-XXX8728 | 2       | 2021 | 4877512 |
| XX-XXX8728 | 2       | 2021 | 4139199 |
| XX-XXX8142 | 1       | 2021 | 1002241 |
| XX-XXX8142 | 2       | 2020 | 524803  |
| XX-XXX8142 | 2       | 2021 | 465617  |
+------------+---------+------+---------+


In [64]:
# save the 941 dataframe!
imperfect_941.to_csv('imperfect_941.csv', index = False)

In [65]:
%%coconut

# read back in the 941 dataframe
imperfect_941 = pd.read_csv('imperfect_941.csv')

imperfect_941 \
    |> select(-_.file_name) \
    |> mutate(ein = _.ein.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \
    |> head(15) \
    |> table

+------------+---------+------+---------+
| ein        | quarter | year | line_27 |
+------------+---------+------+---------+
| XX-XXX4548 | 2       | 2020 | 646831  |
| XX-XXX4548 | 3       | 2020 | 791070  |
| XX-XXX4548 | 4       | 2020 | 578313  |
| XX-XXX4548 | 1       | 2021 | 842800  |
| XX-XXX4548 | 2       | 2021 | 1773100 |
| XX-XXX4548 | 3       | 2021 | 2186100 |
| XX-XXX8728 | 2       | 2020 | 2032059 |
| XX-XXX8728 | 3       | 2020 | 816728  |
| XX-XXX8728 | 4       | 2020 | 654438  |
| XX-XXX8728 | 1       | 2021 | 3235630 |
| XX-XXX8728 | 2       | 2021 | 4877512 |
| XX-XXX8728 | 2       | 2021 | 4139199 |
| XX-XXX8142 | 1       | 2021 | 1002241 |
| XX-XXX8142 | 2       | 2020 | 524803  |
| XX-XXX8142 | 2       | 2021 | 465617  |
+------------+---------+------+---------+


**Pivot**

In [66]:
import pandas as pd

def create_quarter_columns(df):
    # Ensure quarter_year column exists
    if 'quarter_year' not in df.columns:
        df['quarter_year'] = df.apply(lambda x: f"Q{x['quarter']}_{x['year']}", axis=1)
    
    # Pivot
    df_pivot = pd.pivot_table(
        data=df,
        index=['ein'],  # Group by EIN
        columns='quarter_year',
        values='line_27',
        fill_value=pd.NA  # Fill missing values with NA
    )
    
    # Add prefixes to column names
    df_pivot.columns = [f'line27_{col}' for col in df_pivot.columns]
    
    # Reset index to keep 'ein' as a column
    df_pivot = df_pivot.reset_index()
    
    return df_pivot

# Perform the pivot
perfect_941 = create_quarter_columns(imperfect_941.copy())

In [68]:
%%coconut

# preview
perfect_941 \
    |> mutate(ein = _.ein.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \ 
    |> head(15) \
    |> table

+------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+
| ein        | line27_Q1_2021 | line27_Q2_2020 | line27_Q2_2021 | line27_Q3_2020 | line27_Q3_2021 | line27_Q4_2020 | line27_Q4_2021 |
+------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+
| XX-XXX9586 | 277760.0       | 483157.0       | 1532598.0      | 266823.0       | 1696713.0      | nan            | nan            |
| XX-XXX8731 | 4264348.0      | 3429328.0      | 4310776.0      | 870632.0       | 5181663.0      | 243875.0       | nan            |
| XX-XXX6238 | 3040590.0      | 2500000.0      | 3118500.0      | nan            | 3118500.0      | nan            | nan            |
| XX-XXX1098 | 5585824.0      | 3965406.0      | 7976259.0      | 2525304.0      | 8340664.0      | 2259390.0      | nan            |
| XX-XXX1055 | 5595961.0      | 929750.0       | 6969440.0    

**Clean and save the perfect df**

In [69]:
%%coconut

from siuba import \filter

pd.set_option('display.float_format', '{:.2f}'.format)

perfect_941_2 = perfect_941.copy() |> mutate(
    line27_Q1_2021 = _.line27_Q1_2021 / 100,
    line27_Q2_2020 = _.line27_Q2_2020 / 100,
    line27_Q2_2021 = _.line27_Q2_2021 / 100,
    line27_Q3_2020 = _.line27_Q3_2020 / 100,
    line27_Q3_2021 = _.line27_Q3_2021 / 100,
    line27_Q4_2020 = _.line27_Q4_2020 / 100,
    line27_Q4_2021 = _.line27_Q4_2021 / 100
) |> select(
    _.ein,
    _.line27_Q2_2020,
    _.line27_Q3_2020,
    _.line27_Q4_2020,
    _.line27_Q1_2021,
    _.line27_Q2_2021,
    _.line27_Q3_2021
) |> filter(_.ein != '26-4119062')

perfect_941_2.to_csv('perfect_941.csv', index = False)

perfect_941_2 \
    |> mutate(ein = _.ein.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \
    |> head(15) \
    |> table

+------------+----------------+----------------+----------------+----------------+----------------+----------------+
| ein        | line27_Q2_2020 | line27_Q3_2020 | line27_Q4_2020 | line27_Q1_2021 | line27_Q2_2021 | line27_Q3_2021 |
+------------+----------------+----------------+----------------+----------------+----------------+----------------+
| XX-XXX9586 | 4831.57        | 2668.23        | nan            | 2777.6         | 15325.98       | 16967.13       |
| XX-XXX8731 | 34293.28       | 8706.32        | 2438.75        | 42643.48       | 43107.76       | 51816.63       |
| XX-XXX6238 | 25000.0        | nan            | nan            | 30405.9        | 31185.0        | 31185.0        |
| XX-XXX1098 | 39654.06       | 25253.04       | 22593.9        | 55858.24       | 79762.59       | 83406.64       |
| XX-XXX1055 | 9297.5         | 7059.9         | 8120.18        | 55959.61       | 69694.4        | 70819.25       |
| XX-XXX5564 | nan            | 20709.31       | 10467.25       

In [70]:
%%coconut

import pandas as pd
from siuba import *

# load back in the perfect_941 df
perfect_941 = pd.read_csv('perfect_941.csv')

perfect_941 \
    |> mutate(ein = _.ein.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \
    |> head(15) \
    |>  table

+------------+----------------+----------------+----------------+----------------+----------------+----------------+
| ein        | line27_Q2_2020 | line27_Q3_2020 | line27_Q4_2020 | line27_Q1_2021 | line27_Q2_2021 | line27_Q3_2021 |
+------------+----------------+----------------+----------------+----------------+----------------+----------------+
| XX-XXX9586 | 4831.57        | 2668.23        | nan            | 2777.6         | 15325.98       | 16967.13       |
| XX-XXX8731 | 34293.28       | 8706.32        | 2438.75        | 42643.48       | 43107.76       | 51816.63       |
| XX-XXX6238 | 25000.0        | nan            | nan            | 30405.9        | 31185.0        | 31185.0        |
| XX-XXX1098 | 39654.06       | 25253.04       | 22593.9        | 55858.24       | 79762.59       | 83406.64       |
| XX-XXX1055 | 9297.5         | 7059.9         | 8120.18        | 55959.61       | 69694.4        | 70819.25       |
| XX-XXX5564 | nan            | 20709.31       | 10467.25       

# **Step 5 - Join the transactions and perfect 941-x dataframe**

In [74]:
%%coconut

transactions_proc \
    |> select(-_.taxpayer_name) \
    |> mutate(tin = _.tin.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \
    |> head(15) \
    |> table

+------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
| tin        | refund_total | refunds_Q2_2020 | refunds_Q3_2020 | refunds_Q4_2020 | refunds_Q1_2021 | refunds_Q2_2021 | refunds_Q3_2021 | refunds_Q4_2021 |
+------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
| XX-XXX5545 | 160048.23    | 35115.6         | 61591.56        | 0.0             | 63341.07        | 0.0             | 0.0             | 0.0             |
| nan        | 2641.57      | 0.0             | 0.0             | 0.0             | 0.0             | 0.0             | 2641.57         | 0.0             |
| nan        | 54802.05     | 7860.09         | 20082.07        | 26859.89        | 0.0             | 0.0             | 0.0             | 0.0             |
| nan        | 11136.81     | 4297.54         | 4027.46         

In [75]:
%%coconut

from siuba import \filter

# complete the left join between transactions and 941
# and filter out NA EIN values
erc = (
    transactions_proc.merge(perfect_941,
                             how = 'left',
                             left_on = 'tin',
                             right_on = 'ein'
                           )
) |> filter(~_.tin.isna())

# rename line27 columns to erc_claimed
erc = erc \
    |> _.rename(columns = {'line27_Q2_2020': 'erc_claimed_Q2_2020'}) \
    |> _.rename(columns = {'line27_Q3_2020': 'erc_claimed_Q3_2020'}) \
    |> _.rename(columns = {'line27_Q4_2020': 'erc_claimed_Q4_2020'}) \
    |> _.rename(columns = {'line27_Q1_2021': 'erc_claimed_Q1_2021'}) \
    |> _.rename(columns = {'line27_Q2_2021': 'erc_claimed_Q2_2021'}) \
    |> _.rename(columns = {'line27_Q3_2021': 'erc_claimed_Q3_2021'})

# create a filing_total column for summing erc claimed for each quarter
erc = erc \
    |> mutate(
        filing_total = 
        _.erc_claimed_Q2_2020.fillna(0) + 
        _.erc_claimed_Q3_2020.fillna(0) + 
        _.erc_claimed_Q4_2020.fillna(0) + 
        _.erc_claimed_Q1_2021.fillna(0) + 
        _.erc_claimed_Q2_2021.fillna(0) + 
        _.erc_claimed_Q3_2021.fillna(0)
        # + _.erc_claimed_Q4_2021.fillna(0)
    ) |> _.round(2)

# rearrange columns
erc = erc \
    |> select(
        'taxpayer_name', 
        'tin', 
        'refund_total', 
        'filing_total', 
        'refunds_Q2_2020',
        'erc_claimed_Q2_2020',
        'refunds_Q3_2020', 
        'erc_claimed_Q3_2020',
        'refunds_Q4_2020', 
        'erc_claimed_Q4_2020',
        'refunds_Q1_2021',
        'erc_claimed_Q1_2021',
        'refunds_Q2_2021', 
        'erc_claimed_Q2_2021',
        'refunds_Q3_2021', 
        'erc_claimed_Q3_2021'
        # ,'refunds_Q4_2021'
        # ,'erc_claimed_Q4_2021'
)

erc |> select(-_.taxpayer_name) \
    |> mutate(tin = _.tin.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \ 
    |> head(20) \
    |> table_s

+------------+--------------+--------------+-----------------+-----------------+---------------------+-----------------+---------------------+
| tin        | refund_total | filing_total | refunds_Q2_2020 | refunds_Q2_2021 | erc_claimed_Q2_2021 | refunds_Q3_2021 | erc_claimed_Q3_2021 |
+------------+--------------+--------------+-----------------+-----------------+---------------------+-----------------+---------------------+
| XX-XXX5545 | 160048.23    | 54683.21     | 35115.6         | 0.0             | 3919.97             | 0.0             | nan                 |
| XX-XXX8326 | 0.0          | 69251.24     | 0.0             | 0.0             | 14222.04            | 0.0             | 19186.57            |
| XX-XXX0063 | 62612.66     | 196919.52    | 21808.07        | 0.0             | 47908.0             | 38857.24        | 38440.5             |
| XX-XXX0665 | 0.0          | 76072.77     | 0.0             | 0.0             | 20583.0             | 0.0             | 25209.67            |

In [37]:
# save the final erc file
erc.to_csv('erc_01-30-25.csv', index = False)

In [78]:
%%coconut

import pandas as pd

erc = pd.read_csv('erc_01-30-25.csv')

erc |> select(-_.taxpayer_name) \
    |> mutate(tin = _.tin.str[-4:].str.rjust(9, 'X').str.replace(r'(^.{2})', r'\1-', regex=True)) \
    |> head(25) \
    |> table_s

+------------+--------------+--------------+-----------------+-----------------+---------------------+-----------------+---------------------+
| tin        | refund_total | filing_total | refunds_Q2_2020 | refunds_Q2_2021 | erc_claimed_Q2_2021 | refunds_Q3_2021 | erc_claimed_Q3_2021 |
+------------+--------------+--------------+-----------------+-----------------+---------------------+-----------------+---------------------+
| XX-XXX5545 | 160048.23    | 54683.21     | 35115.6         | 0.0             | 3919.97             | 0.0             | nan                 |
| XX-XXX8326 | 0.0          | 69251.24     | 0.0             | 0.0             | 14222.04            | 0.0             | 19186.57            |
| XX-XXX0063 | 62612.66     | 196919.52    | 21808.07        | 0.0             | 47908.0             | 38857.24        | 38440.5             |
| XX-XXX0665 | 0.0          | 76072.77     | 0.0             | 0.0             | 20583.0             | 0.0             | 25209.67            |